In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as tf_text

tf.get_logger().setLevel('ERROR')

In [2]:
data = pd.read_feather('/root/autodl-nas/pretrain_data.ftr')
data = data[data['channels'].apply(len) != 0]
data['channels'] = data['channels'].apply(lambda x: x.split(','))
data

,datetime,content,channels
0,2018-10-08 10:22:47,【庞大陷危局 接盘者宝能现身】继恒大145亿元入股广汇汽车后，又一家资本巨头准备接手经销商集...,"[公司, A股]"
1,2018-10-09 14:20:43,【液氯价格较昨日涨25.96%】液氯价格今日报255.01元/吨，较昨日涨25.96%；钒铁...,[行业]
2,2018-10-09 14:22:27,【华为表示已收到印度政府参与5G试验邀请】据报道，目前华为已经收到印度政府的邀请，将参与该国...,[公司]
3,2018-10-09 14:24:32,【统计局：2017年国家财政科学技术支出8383亿元 增长8%】国家统计局9日披露，2017...,[宏观]
4,2018-10-09 14:25:25,【银龙股份：公司出口的大部分产品退税率均从5%提高到9%】银龙股份在互动平台上表示，财政部和...,"[公司, A股]"
...,...,...,...
1076899,2022-12-07 14:29:27,【金龙羽：公司固态电池研发项目未使用钠材料】金龙羽(002882)在互动平台表示，公司固态电...,[公司]
1076900,2022-12-07 14:29:43,【建业地产午后一度升15% 已汇出2023年11月到期债券应计利息】建业地产(00832...,[公司]
1076901,2022-12-07 14:30:22,【天齐锂业：澳大利亚奎纳纳年产4.8万吨电池级氢氧化锂工厂已具备连续生产运行能力】天齐锂业1...,[公司]
1076902,2022-12-07 14:31:47,韩国KOSPI指数收盘下跌0.42%，报2383.19点。,[国际]


In [3]:
labels = []
for i in data['channels']:
    labels += i
classes = pd.value_counts(labels)
classes = dict([(j, i) for i, j in enumerate(classes.index.tolist())])
classes

{'公司': 0,
 '国际': 1,
 '市场': 2,
 '焦点': 3,
 'A股': 4,
 '疫情': 5,
 '其他': 6,
 '宏观': 7,
 '行业': 8,
 '观点': 9,
 '数据': 10,
 '央行': 11}

In [4]:
def make_labels(x):
    x = [classes[i] for i in x]
    y = np.zeros(12)
    y[x] += 1
    return y.astype(int)
data['labels'] = data['channels'].apply(make_labels)
data

,datetime,content,channels,labels
0,2018-10-08 10:22:47,【庞大陷危局 接盘者宝能现身】继恒大145亿元入股广汇汽车后，又一家资本巨头准备接手经销商集...,"[公司, A股]","[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
1,2018-10-09 14:20:43,【液氯价格较昨日涨25.96%】液氯价格今日报255.01元/吨，较昨日涨25.96%；钒铁...,[行业],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
2,2018-10-09 14:22:27,【华为表示已收到印度政府参与5G试验邀请】据报道，目前华为已经收到印度政府的邀请，将参与该国...,[公司],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,2018-10-09 14:24:32,【统计局：2017年国家财政科学技术支出8383亿元 增长8%】国家统计局9日披露，2017...,[宏观],"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
4,2018-10-09 14:25:25,【银龙股份：公司出口的大部分产品退税率均从5%提高到9%】银龙股份在互动平台上表示，财政部和...,"[公司, A股]","[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...
1076899,2022-12-07 14:29:27,【金龙羽：公司固态电池研发项目未使用钠材料】金龙羽(002882)在互动平台表示，公司固态电...,[公司],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1076900,2022-12-07 14:29:43,【建业地产午后一度升15% 已汇出2023年11月到期债券应计利息】建业地产(00832...,[公司],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1076901,2022-12-07 14:30:22,【天齐锂业：澳大利亚奎纳纳年产4.8万吨电池级氢氧化锂工厂已具备连续生产运行能力】天齐锂业1...,[公司],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1076902,2022-12-07 14:31:47,韩国KOSPI指数收盘下跌0.42%，报2383.19点。,[国际],"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [5]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, train_size=0.99)

In [ ]:
batchsize = 16 * 3
tf_train_data = tf.data.Dataset.from_tensor_slices(
    (train_data['content'], train_data['labels'].values.tolist())
).batch(batchsize).prefetch(tf.data.AUTOTUNE)
tf_test_data = tf.data.Dataset.from_tensor_slices(
    (test_data['content'], test_data['labels'].values.tolist())).batch(batchsize)

In [7]:
strategy = tf.distribute.MirroredStrategy()

In [8]:
with strategy.scope():
    preprocessor = hub.KerasLayer('bert-model/preprocessor/')
    bert_model = hub.KerasLayer('bert-model/model/')
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
    emb_vec = bert_model(preprocessor(text_input))['pooled_output']
    output_layer = tf.keras.layers.Dense(len(classes), activation='sigmoid')(emb_vec)
    model = tf.keras.models.Model(text_input, output_layer)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(), 
        loss=tf.keras.losses.BinaryCrossentropy(), 
        metrics=['ACC', 'AUC']
    )

In [ ]:
import os

checkpoint_path = "/root/training_checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
# Define a function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

# Put all the callbacks together.
callbacks = [
    cp_callback, 
    tf.keras.callbacks.LearningRateScheduler(decay)
]

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
model.fit(tf_train_data, epochs=10, validation_data=tf_test_data, callbacks=callbacks)

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['input_1[0][0]']                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [14]:
finbert = model.get_layer('keras_layer_1')
finbert = tf.keras.models.Model(finbert.input, finbert.output)
finbert.save('/root/autodl-nas/FinBERT')

2022-12-08 00:24:18.556697: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
